<b> One-Layer Atmosphere Model - 2XCO2, Water-Wapor feedback </b><br>
Reference: Walter A. Robinson, Modeling Dynamic Climate Systems

In [1]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-dark')

In [2]:
# Step size
dt = 0.01

# Set up simulation time
tmin = 0
tmax = 10
t = np.arange(tmin, tmax + dt, dt)
n = len(t)

# Seconds per year
seconds_per_year = 365 * 24 * 60 * 60

In [3]:
# Albedo
albedo = 0.30

# Emissivity
e =  0.85
e_0 = 0.75
t_0 = 273.15 # K
t_1 = 5423 # K
default_e = e

# Absorption
atmospheric_abs = 0.1

# Stefan-Boltzmann constant
sigma = 5.6696e-8 # W/m^2*K^4

# Solar constant 
solar_const = 1367 # W/m^2
default_solar_const = solar_const # W/m^2

# Density of water
water_density = 1000 # kg/m^3

# Depth of the mixed layer
depth_mixed_layer = 50 # m

# Specific heat capacity of water
spec_heat_water = 4218 # J/kg*K

# Specific heat capacity of atmosphere
spec_heat_atm = 1004 # J/kg*K

# Gravity accelleration
g = 9.81 # m/s^2

# Atmospheric pressure
atm_press = 1e5; # Pa (kg/m*s^2)

# Atmospheric mass
mass_atm = atm_press/g;  # kg/m^2

In [4]:
# Heat capacity water
heat_capacity_water = water_density * depth_mixed_layer * spec_heat_water # J/K

# Heat capacity atmosphere
heat_capacity_atm = mass_atm * spec_heat_atm # J/K

In [5]:
# 1: No 2xCO2, 2: 2xCO2 - No Feedbacks, 3: 2xCO2 + Water-vapor feedback
models = {"model_1": {"desc": "No 2xCO2", 
                      "stronger_sun": 0, 
                      "h2o_feedback": 0, 
                      "T_final": 0},
          "model_2": {"desc": "2xCO2 - No Feedbacks", 
                      "stronger_sun": 1, 
                      "h2o_feedback": 0 , 
                      "T_final": 0},
          "model_3": {"desc": "2xCO2 + Water-vapor feedback", 
                      "stronger_sun": 1, 
                      "h2o_feedback": 1 , 
                      "T_final": 0}}

In [6]:
# Store the final surface temperature of each model
for model_name, model in models.items():
    # Initial temperature
    ts = 273.15 # K
    ta = 273.15 # K
    
    if model["stronger_sun"]:
        # Stronger sun emulate the doubling of CO2 (2xCO2)
        solar_const = default_solar_const * 1.02
    else:
        solar_const = default_solar_const

    if model["h2o_feedback"]:
        e = default_e

    for _ in range(n):
        if model["h2o_feedback"]:
            e = max(e_0 + 0.1 * t_1 * (1/t_0 - 1/ts), 0)
          
        # Inflows (Solar to earth, IR down) - Surface
        solar_to_earth = (solar_const/4 * (1 - albedo) * seconds_per_year) * (1 - atmospheric_abs)
        ir_down = sigma * e * ta ** 4 * seconds_per_year
        
        # Outflows (IR) - Surface
        ir = sigma * ts ** 4 * seconds_per_year
        
        # Calculate the temperature - Surface
        energy_ts = ts * heat_capacity_water + (solar_to_earth + ir_down - ir) * dt
        ts = energy_ts/heat_capacity_water
        
        # Inflows (solar to atmosphere, IR) - Atmosphere
        solar_to_atm = (solar_const/4 * (1 - albedo) * seconds_per_year) * atmospheric_abs
        
        # Outflows (IR down, IR to space) - Atmosphere
        ir_to_space = ir * (1 - e) + ir_down
        
        # Calculate the temperature - Atmosphere
        energy_ta = ta * heat_capacity_atm + (ir + solar_to_atm - ir_down - ir_to_space) * dt
        ta = energy_ta/heat_capacity_atm
    models[model_name]["T_final"] = ts

In [7]:
# Model: No 2xCO2
model_1 = models["model_1"]["T_final"]
print(f"No 2xCO2, T={round(model_1)}K")

No 2xCO2, T=289K


In [8]:
# Model: 2xCO2, No feedbacks
model_2 = models["model_2"]["T_final"]
print(f"2xCO2 + No feedbacks, T={round(model_2)}K")

2xCO2 + No feedbacks, T=290K


In [9]:
# Model: 2xCO2 + No feedbacks
model_3 = models["model_3"]["T_final"]
print(f"2xCO2 + Water-vapor feedback, T={round(model_3)}K")

2xCO2 + Water-vapor feedback, T=291K


In [10]:
# Temperature changes in response to 2xCO2
delta_t2 = model_2 - model_1
delta_t3 = model_3 - model_1
print(f"2xCO2 + No feedbacks, ∆T={round(delta_t2, 2)}")
print(f"2xCO2 + Water-vapor feedback, ∆T={round(delta_t3, 2)}")

2xCO2 + No feedbacks, ∆T=1.43
2xCO2 + Water-vapor feedback, ∆T=2.31


In [11]:
# Temperature change in response to the "Water-wapor feedback
delta_H2O = delta_t3 - delta_t2
print(f"Water-vapor feedback, ∆T={round(delta_H2O, 2)}")

Water-vapor feedback, ∆T=0.89


In [12]:
# Fraction of the total temperature change that is due to the water-wapor feedback (Gain)
gain_H2O = delta_H2O/delta_t3
print(f"Water-vapor feedback, g={round(gain_H2O, 2)}")

Water-vapor feedback, g=0.38
